# DG method to solve elliptic problem 

## Model problem 

$$
\begin{equation}
    \begin{aligned}
        -\nabla \cdot (\mathbf{K} \nabla p) + \alpha p &= f \qquad \text{ in } \Omega, \\
        p &= g_D \qquad \text{on } \Gamma_D, \\
        \mathbf{K} \nabla p \cdot \mathbf{n} &= g_N \qquad \text{on } \Gamma_N.
    \end{aligned}
\end{equation}
$$

## Variational formulation

$$
a_{\epsilon}(u,v) = \sum_{E \in \mathcal{E}_h} \int_E \mathbf{K} \nabla u \cdot \nabla v + \int_{\Omega} \alpha uv 
-  \sum_{e \in \Gamma_h \bigcup \Gamma_D} \int_e \{ \mathbf{K}  \nabla u \cdot \mathbf{n}_e \} [v] 
+ \epsilon \sum_{e \in \Gamma_h \bigcup \Gamma_D} \int_e \{ \mathbf{K}  \nabla v \cdot \mathbf{n}_e \} [u] 
+ \frac{\sigma}{h} \sum_{e \in \Gamma_h \bigcup \Gamma_D}  \int_e [u][v]
$$

$$
L(v) = \int_{\Omega} fv +  \sum_{e\in \Gamma_D} \int_e (\epsilon \mathbf{K} \nabla v \cdot \mathbf{n}_e + \frac{\sigma}{h}v )g_D + \sum_{e\in \Gamma_N} \int_e vg_N
$$

In [1]:
from ngsolve import *
from ngsolve.webgui import Draw

In [14]:
# Define important parameters
epi = -1
sigma = 10
order = 0
alpha = 1
mh=0.5
f = (2+alpha) * sin(x) * sin(y)
dirichlet_bnd = "left|right|top"
gD = sin(x) * sin(y)
# gD = CoefficientFunction([sin(x) * sin(y) if bc!="bottom" else 0 for bc in mesh.GetBoundaries()])
neumann_bnd = 'bottom'
gN = CoefficientFunction(-sin(x) * cos(y))  # 下边界取值


In [15]:
# Define the mesh
mesh = Mesh(unit_square.GenerateMesh(maxh=mh))
Draw(mesh)
print(mesh.Boundaries(neumann_bnd))


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

In [16]:
# Define the DG space 
# fes = L2(mesh,order=order,dgjumps=True, dirichlet = dirichlet_bnd)
fes = L2(mesh,order=order,dgjumps=True)
u,v = fes.TnT()

In [17]:
# Define the jumps and the averages
jump_u = u - u.Other()
jump_v = v - v.Other()
n = specialcf.normal(2)
mean_dudn = 0.5*n*(grad(u)+grad(u.Other()))
mean_dvdn = 0.5*n*(grad(v)+grad(v.Other()))
h = specialcf.mesh_size           

In [18]:
# Define bilinear form
diffusion = grad(u)*grad(v)*dx + alpha*u*v*dx \
            + (epi * mean_dvdn * jump_u- mean_dudn * jump_v)*dx(skeleton=True) \
            + sigma/h* jump_u*jump_v*dx(skeleton=True) \
            + (epi * grad(v).Trace()*n*u - grad(u).Trace()*n*v)*ds(skeleton=True,definedon=mesh.Boundaries(dirichlet_bnd))  \
            + sigma/h*u*v*ds(skeleton=True,definedon=mesh.Boundaries(dirichlet_bnd))

# diffusion =  sigma/h* jump_u*jump_v*dx(skeleton=True) 
        

# diffusion = grad(u)*grad(v)*dx + alpha*u*v*dx \
#             + (epi * mean_dvdn * jump_u - mean_dudn * jump_v)*dx(skeleton=True) \
#             + sigma/h* jump_u*jump_v*dx(skeleton=True) \
#             + (epi * grad(v)*n*u - grad(u)*n*v)*ds(skeleton=True)  \
#             + sigma/h*u*v*ds(skeleton=True)
a_epi = BilinearForm(diffusion).Assemble()

In [19]:
print(a_epi.mat)

Row 0:   0: 20   5: -20
Row 1:   1: 20   4: -20
Row 2:   2: 20   4: -20
Row 3:   3: 20   5: -20
Row 4:   1: -20   2: -20   4: 60   5: -20
Row 5:   0: -20   3: -20   4: -20   5: 60



In [7]:
# Define the right hand side
rl = f * v*dx \
    + (epi*grad(v).Trace()*n + sigma/h*v)*gD*ds(skeleton=True,definedon=mesh.Boundaries(dirichlet_bnd)) \
    + v*gN*ds(skeleton=True,definedon=mesh.Boundaries(neumann_bnd))
# rl = f * v*dx + (epi*grad(v)*n + sigma/h*v)*gD*ds(skeleton=True)
F = LinearForm(rl).Assemble()
# print(F.vec.Norm())


In [8]:
# Solve
gfu = GridFunction(fes,name="uDG")
gfu.vec.data = a_epi.mat.Inverse()*F.vec
Draw(gfu,mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

In [9]:
Draw(gD,mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

In [10]:
print(mesh.GetBoundaries())

('bottom', 'right', 'top', 'left')


In [11]:
# L2误差计算
u_exact = sin(x)*sin(y)
err = sqrt(Integrate((gfu - u_exact)**2, mesh))
print("L2 error =", err)

L2 error = 0.0003678238499591595


In [101]:
print("Neumann RHS contribution =",
      Integrate(gN, mesh, definedon=mesh.Boundaries(neumann_bnd)))

Neumann RHS contribution = -0.4596976977252602


-0.45969769413186023


In [45]:
for edge in mesh.edges:
    print ("dofs = ", fes.GetDofNrs(edge)) 


dofs =  ()
dofs =  ()
dofs =  ()
dofs =  ()
dofs =  ()
dofs =  ()
dofs =  ()
dofs =  ()
dofs =  ()
dofs =  ()
dofs =  ()
dofs =  ()
dofs =  ()


In [68]:
help(ds)

Help on DifferentialSymbol in module ngsolve.comp object:

class DifferentialSymbol(pybind11_builtins.pybind11_object)
 |  Method resolution order:
 |      DifferentialSymbol
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |
 |  Methods defined here:
 |
 |  __call__(...)
 |      __call__(self: ngsolve.comp.DifferentialSymbol, definedon: Optional[Union[ngsolve.comp.Region, str]] = None, element_boundary: bool = False, element_vb: ngsolve.comp.VorB = <VorB.VOL: 0>, skeleton: bool = False, bonus_intorder: int = 0, intrules: dict[ngsolve.fem.ET, ngsolve.fem.IntegrationRule] = {}, deformation: ngsolve.comp.GridFunction = None, definedonelements: pyngcore.pyngcore.BitArray = None) -> ngsolve.comp.DifferentialSymbol
 |
 |  __init__(...)
 |      __init__(self: ngsolve.comp.DifferentialSymbol, arg0: ngsolve.comp.VorB) -> None
 |
 |  ----------------------------------------------------------------------
 |  Static methods inherited from pybind11_builtins.pybind11_object:
 |
 |

In [111]:
help(specialcf)

Help on SpecialCFCreator in module ngsolve.fem object:

class SpecialCFCreator(pybind11_builtins.pybind11_object)
 |  Method resolution order:
 |      SpecialCFCreator
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |
 |  Methods defined here:
 |
 |  EdgeCurvature(...)
 |      EdgeCurvature(self: ngsolve.fem.SpecialCFCreator, dim: int) -> ngfem::CoefficientFunction
 |
 |      EdgeCurvature
 |      space-dimension must be provided
 |
 |  EdgeFaceTangentialVectors(...)
 |      EdgeFaceTangentialVectors(self: ngsolve.fem.SpecialCFCreator, dim: int) -> ngfem::CoefficientFunction
 |
 |      EdgeFaceTangentialVectors
 |      space-dimension must be provided
 |
 |  JacobianMatrix(...)
 |      JacobianMatrix(*args, **kwargs)
 |      Overloaded function.
 |
 |      1. JacobianMatrix(self: ngsolve.fem.SpecialCFCreator, dim: int) -> ngfem::CoefficientFunction
 |
 |      Jacobian matrix of transformation to physical element
 |      space-dimension must be provided
 |
 |      2. 

In [116]:
[sin(x) * sin(y) if bc!="bottom" else 0 for bc in mesh.GetBoundaries()]

[0,

In [119]:
Integrate(v * ds('bottom'), mesh)

NgException: cannot evaluate ProxyFunction without userdata